### 1.1 Коллекция


Для экспериментов Вам предоставляется коллекция документов неизвестного (вам) происхождения.
Каждый документ коллекции представлен в виде txt файла, в неизвестной кодировке, с преобладанием латинских символов. Документы даны как есть, без специальной структуры. Ознакомьтесь, пожалуйста, с содержимым нескольких из них, чтобы понять природу их происхождения. Архив с документами можно не распаковывать, а читать напрямую из кода. Пример чтения архива на питоне – в ноутбуке рядом с заданием. 

Пример чтения архива на питоне:

In [1]:
import tarfile

def read_collection():
    docs = {}
    with tarfile.open('./collection.tar.bz2', 'r:bz2') as tf:
            for ti in tf:
                byte_content = tf.extractfile(ti).read()
                docs[ti.name] = byte_content
    return docs

docs = read_collection()
print(len(docs))
print(docs['172672.txt'])

5258
b'#include <fstream>\r\nusing namespace std;\r\n\r\nvoid main(){\r\n\tint count = 0;\r\n\tfstream in("input.txt");\r\n\tofstream out("output.txt");\r\n\r\n\tin >> count;\r\n\tint* arr = new int[count + 1];\r\n\tint cur = 0;\r\n\tfor (int i = 1; i <= count, in >> cur; i++){// i=1\r\n\t\tarr[i] = cur;\r\n\t}\r\n\r\n\tbool flag = true;\r\n\tfor (int i = 1; 2*i <= count; i++){  \r\n\t\tif (2 * i + 1 <= count){\r\n\t\t\tif (arr[i] > arr[2 * i + 1])\r\n\t\t\t\tflag = false;\r\n\t\t}\r\n\t\tif (arr[i] > arr[2 * i])\r\n\t\t\tflag = false;\r\n\t}\r\n\r\n\tif (flag){\r\n\t\tout << "Yes";\r\n\t}\r\n\telse\r\n\t\tout << "No";\r\n\tin.close();\r\n\tout.close();\r\n}'


### 1.2 Точные дубли
Прочитайте документы и проведите простейшую нормализацию содержимого документов, которая не меняет его сути. Например, следует сделать как минимум: разбор кодировки, удаление BOM, замена табуляций и переводов строк на пробелы, удаление последовательностей пробельных символов. Но только пожалуйста никаких, прости господи, лемматизаций и стемминга.
	
Рассчитайте свою любимую многобитную хеш-функцию для нормализованных документов и найдите точные дубли с помощью известной структуры данных.
	
Оцените какие группы дубликатов получилось найти: количество групп, минимальный, максимальный и средний размер.
	
Оставьте из каждой группы один документ для дальнейших заданий.

In [2]:
import chardet
import re
from hashlib import md5

In [3]:
def normalize(content):
    content = re.sub('[\s]+', ' ', content)
    return content

In [4]:
def remove_spaces(content):
    content = re.sub('[\s]+', '', content)
    return content

In [5]:
encodings = set()
documents = dict()
hashes = dict()
for doc, content in docs.items():
    encoding = chardet.detect(content)['encoding']
    encodings.add(encoding)
    documents[doc] = normalize(content.decode(encoding))
    hash_doc = md5(remove_spaces(documents[doc]).encode('utf-8')).hexdigest()
    if hash_doc in hashes:
        hashes[hash_doc].append(doc)
    else:
        hashes[hash_doc] = [doc]

In [6]:
encodings

{'ISO-8859-1',
 'ISO-8859-9',
 'MacCyrillic',
 'UTF-16',
 'UTF-8-SIG',
 'Windows-1252',
 'ascii',
 'utf-8',
 'windows-1251'}

In [7]:
duplicates = []
sizes = []
for value in hashes.values():
    if len(value) > 1:
        duplicates.append(value)
        sizes.append(len(value))

In [8]:
import numpy as np

In [9]:
np.mean(sizes)

2.3157894736842106

In [10]:
np.min(sizes)

2

In [11]:
np.max(sizes)

9

In [12]:
np.unique(sizes)

array([2, 3, 4, 5, 9])

In [13]:
len(sizes)

171

In [14]:
len(hashes)

5033

### 1.2 Ground truth
В файле ground_truth.tsv.bz2 содержится результат работы «тяжелого» алгоритма сравнения документов на дубли для всех пар документов. Результат для одной пары документов — это число от 0 до 1. Отсутствие пары в архиве означает, что их схожесть равна нулю. Далее будем называть содержимое этого файла -  ground truth.

Пример чтения архива на питоне – в ноутбуке рядом с заданием. Для иноязычных, далее в этом параграфе идет описание формата файла. Каждая строка файла описывает список потенциальных дубликатов для одного из документов коллекции. Строка файла разбита на колонки символом табуляции. В первой колонке записано имя документа, для которого далее следует список дублей. Далее в каждой колонке записано имя файла и оценка задублированности через символ = (равно). 

In [15]:
import bz2
import time


def parse_single_result(note):
    return note.split('=')
    
def parse_ground_truth_line(line):
    items = line.decode().strip().split('\t')
    doc_id = items[0]
    result = [parse_single_result(item) for item in items[1:]]
    return doc_id, {key: float(value) for key, value in result}

def read_ground_truth():
    ground_truth = {}
    bz_file = bz2.BZ2File('./ground_truth.tsv.bz2')
    items = (parse_ground_truth_line(line) for line in bz_file.readlines())
    return dict(items)

ground_truth = read_ground_truth()      
for doc_id in ground_truth:
    for other_id in ground_truth[doc_id]:
        assert ground_truth[other_id][doc_id] ==  ground_truth[doc_id][other_id]
        
print('Ground truth size', len(ground_truth))

Ground truth size 5258


Для каждой пары документов, которые Вы посчитали дублем в пункте 1, убедитесь, что данная пара есть в ground truth и что значение равно единице (aka полные дубли).

Так как точные дубли не представляют интереса для поиска неточных дублей исключите из ground truth пары, найденные в пункте 1.

In [16]:
pairs = []
for d in duplicates:
    for i in range(len(d)):
        for j in range(len(d)):
            if i != j:
                pairs.append((d[i], d[j]))

In [17]:
for first, second in pairs:
    assert ground_truth[first][second] == 1
    ground_truth[first].pop(second)

In [18]:
for d in duplicates:
    for doc in d[1:]:
        del docs[doc]
        del documents[doc]
        del ground_truth[doc]

In [19]:
len(documents)

5033

In [20]:
len(ground_truth)

5033

### 1.4 MinHash
Разбейте документ на слова с учетом специфики содержимого документов. Например, разделения по пробелам может быть недостаточно, а пунктуация в этой задаче намного важнее, чем при обработке обычных веб-страниц. 

Рассчитайте MinHash описанный на лекции для всех документов. Разбейте документы по дублям из расчета, что дубли имеют одинаковый MinHash.

Реализуйте поиск ближайших документов для произвольного количества MinHash хеш-функций (k). Отсортируйте документы кандидаты по убыванию степени «похожести» – доле совпавших хешей из k.

Реализованный алгоритм поиска должен быть заметно быстрее, чем линейный проход по всем документам коллекции. Проектируйте решение так, чтобы при появлении нового документа, можно было быстро найти для него кандидаты дубли, а также добавить в базу.

In [21]:
import random

In [22]:
random.seed(42)
coeffs = []
mod = 2**61 - 1

In [23]:
for _ in range(200):
    coeffs.append(random.randint(0, mod - 1))

In [24]:
import nltk

In [26]:
documents['124830.txt']

'#include <fstream> #include <set> int main() { std::ifstream fin("input.txt"); std::ofstream fout("output.txt"); std::set<int> s; int x; while (fin >> x) { s.insert(x); } long long sum = 0; for (std::set<int>::const_iterator it = s.begin(); it != s.end(); ++it) { sum += *it; } fout << sum << \'\\n\'; return 0; } '

In [58]:
tokens = nltk.word_tokenize(documents['124830.txt'])
tokens

['#',
 'include',
 '<',
 'fstream',
 '>',
 '#',
 'include',
 '<',
 'set',
 '>',
 'int',
 'main',
 '(',
 ')',
 '{',
 'std',
 ':',
 ':ifstream',
 'fin',
 '(',
 '``',
 'input.txt',
 "''",
 ')',
 ';',
 'std',
 ':',
 ':ofstream',
 'fout',
 '(',
 '``',
 'output.txt',
 "''",
 ')',
 ';',
 'std',
 ':',
 ':set',
 '<',
 'int',
 '>',
 's',
 ';',
 'int',
 'x',
 ';',
 'while',
 '(',
 'fin',
 '>',
 '>',
 'x',
 ')',
 '{',
 's.insert',
 '(',
 'x',
 ')',
 ';',
 '}',
 'long',
 'long',
 'sum',
 '=',
 '0',
 ';',
 'for',
 '(',
 'std',
 ':',
 ':set',
 '<',
 'int',
 '>',
 ':',
 ':const_iterator',
 'it',
 '=',
 's.begin',
 '(',
 ')',
 ';',
 'it',
 '!',
 '=',
 's.end',
 '(',
 ')',
 ';',
 '++it',
 ')',
 '{',
 'sum',
 '+=',
 '*',
 'it',
 ';',
 '}',
 'fout',
 '<',
 '<',
 'sum',
 '<',
 '<',
 "'\\n",
 "'",
 ';',
 'return',
 '0',
 ';',
 '}']

In [25]:
from tqdm import tqdm

In [26]:
def norm(words):
    dc = dict()
    ws = []
    cnt = 0
    for word in words:
        if word in dc:
            ws.append(dc[word])
        else:
            cnt += 1
            dc[word] = f"word{cnt}"
            ws.append(f"word{cnt}")
    return ws

In [61]:
norm(nltk.word_tokenize(documents['124830.txt']))

['word1',
 'word2',
 'word3',
 'word4',
 'word5',
 'word1',
 'word2',
 'word3',
 'word6',
 'word5',
 'word7',
 'word8',
 'word9',
 'word10',
 'word11',
 'word12',
 'word13',
 'word14',
 'word15',
 'word9',
 'word16',
 'word17',
 'word18',
 'word10',
 'word19',
 'word12',
 'word13',
 'word20',
 'word21',
 'word9',
 'word16',
 'word22',
 'word18',
 'word10',
 'word19',
 'word12',
 'word13',
 'word23',
 'word3',
 'word7',
 'word5',
 'word24',
 'word19',
 'word7',
 'word25',
 'word19',
 'word26',
 'word9',
 'word15',
 'word5',
 'word5',
 'word25',
 'word10',
 'word11',
 'word27',
 'word9',
 'word25',
 'word10',
 'word19',
 'word28',
 'word29',
 'word29',
 'word30',
 'word31',
 'word32',
 'word19',
 'word33',
 'word9',
 'word12',
 'word13',
 'word23',
 'word3',
 'word7',
 'word5',
 'word13',
 'word34',
 'word35',
 'word31',
 'word36',
 'word9',
 'word10',
 'word19',
 'word35',
 'word37',
 'word31',
 'word38',
 'word9',
 'word10',
 'word19',
 'word39',
 'word10',
 'word11',
 'word30',
 'word

In [40]:
def min_hash(content, k, n=2, tokenize=True, remove_words=False, normlz=False, delim=False):
    if delim:
        content = re.sub(r'[\.:;\'\"&*^%$#@!<>?(){}=+-]', '', content)
    if remove_words:
        content = re.sub('[\w\d]+', '', content)
    if tokenize:
        tokens = nltk.word_tokenize(content)
    else:
        tokens = content.split()
    if normlz:
        tokens = norm(tokens)
    mhsh = [mod] * k
    for grams in nltk.ngrams(tokens, n):
        cur = hash(grams)
        for i in range(k):
            val = (cur * coeffs[2 * i] + coeffs[2 * i + 1]) % mod
            mhsh[i] = min(mhsh[i], val)
    return mhsh

Оцените качество подбора дубликатов при различных значениях k. Например, для k равных 1, 10, 50, 100. Для оценки качества воспользуйтесь списком ground truth. В качестве метрики качества предлагается использовать nDCG@10 где релевантностью выступает оценка задублированности из ground truth. Для кандидатов, которых нет в ground truth оценку задублированности стоит считать равной нулю. 


Пример рассчета ndcg10 на питоне.

In [28]:
import numpy as np
np.random.seed(42)

def dcg_score(relevs, p):
    score = 0
    for position, relev in enumerate(relevs[:p]):
        score += (2.0 ** relev - 1) / np.log2(position + 2)
    return score

def idcg10_single_doc(ref_doc_id, ground_truth):
    return dcg_score(sorted(ground_truth[ref_doc_id].values(), reverse=True), 10)

def dcg10_single_doc(ref_doc_id, candidates_doc_ids, ground_truth):
    assert len(set(candidates_doc_ids)) == len(candidates_doc_ids), "All candidates must be different"    
    return dcg_score([ground_truth[ref_doc_id].get(doc_id, 0.0) for doc_id in candidates_doc_ids], 10)

def ndcg10_single_doc(ref_doc_id, candidates_doc_ids, ground_truth):
    # Полученный вашим алгоритмом dcg10
    dcg = dcg10_single_doc(ref_doc_id, candidates_doc_ids, ground_truth)
    
    # Идеальный dcg10 для этого документа
    # чтобы ускорить, стоит предпросчитать ответы
    idcg = idcg10_single_doc(ref_doc_id, ground_truth)
    
    return dcg / idcg if idcg > 1e-10 else 0.0

def ndcg10_stats(title, doc_to_candidates, ground_truth):    
    scores = []
    for doc_id, candidates in doc_to_candidates.items():
        scores.append(ndcg10_single_doc(doc_id, candidates, ground_truth))
    print(title)
    print('  Average:\t', np.mean(scores))
    print('  Median: \t', np.median(scores))
    print('  Std Dev:\t', np.std(scores))
    print('  Minimum:\t', np.min(scores))
    print('  Maximum:\t', np.max(scores))

best_for_172672 = [key for key, value in sorted(ground_truth['172672.txt'].items(), key=lambda x: -x[1])]
best_for_172672_shuffled = np.array(best_for_172672[:10])
np.random.shuffle(best_for_172672_shuffled)
assert ndcg10_single_doc('172672.txt', best_for_172672, ground_truth) == 1.0
assert ndcg10_single_doc('172672.txt', best_for_172672[:10], ground_truth) == 1.0
assert 0.0 < ndcg10_single_doc('172672.txt', best_for_172672_shuffled, ground_truth) < 1.0
assert 0.0 < ndcg10_single_doc('172672.txt', best_for_172672[:9], ground_truth) < 1.0
assert 0.0 < ndcg10_single_doc('172672.txt', best_for_172672[10:], ground_truth) < 1.0
assert ndcg10_single_doc('172672.txt', [], ground_truth) == 0.0
assert ndcg10_single_doc('172672.txt', ['unknown.txt'], ground_truth) == 0.0
assert ndcg10_single_doc('172672.txt', np.random.choice(list(ground_truth), size=10, replace=False), ground_truth) == 0

ndcg10_stats("Ndcg@10 demo", {'172672.txt': best_for_172672, '124830.txt': []}, ground_truth)

Ndcg@10 demo
  Average:	 0.5
  Median: 	 0.5
  Std Dev:	 0.5
  Minimum:	 0.0
  Maximum:	 1.0


In [41]:
def get_nearest(documents, k, n=2, tokenize=True, remove_words=False, normlz=False, delim=False):
    inv_index = [{} for _ in range(k)]
    hshs = {}
    for doc, content in tqdm(documents.items()):
        hshs[doc] = min_hash(content, k, n, tokenize, remove_words, normlz, delim)
        for i in range(k):
            if hshs[doc][i] in inv_index[i]:
                inv_index[i][hshs[doc][i]].append(doc)
            else:
                inv_index[i][hshs[doc][i]] = [doc]
    res = {}
    for i in tqdm(range(k)):
        for doc in documents.keys():
            for other in inv_index[i].get(hshs[doc][i], []):
                if other != doc:
                    dct = res.setdefault(doc, {})
                    dct[other] = dct.get(other, 0.0) + 1 / k
    res_copy = res.copy()
    for doc, others in res_copy.items():
        res[doc] = [key for key, _ in sorted(others.items(), key=lambda x: -x[1])][:10]
    ndcg10_stats(f"k = {k}, {n}-grams:", res, ground_truth)

# Разбиение на слова, при котором знаки пунктуации являются токенами: 

In [42]:
get_nearest(documents, 1, 2)

100%|██████████| 1/1 [00:00<00:00,  1.70it/s]


k = 1, 2-grams:
  Average:	 0.07216541306111947
  Median: 	 0.0
  Std Dev:	 0.14466372695805832
  Minimum:	 0.0
  Maximum:	 1.0


In [43]:
get_nearest(documents, 10, 2)

100%|██████████| 10/10 [00:09<00:00,  1.02it/s]


k = 10, 2-grams:
  Average:	 0.30513997374056306
  Median: 	 0.30048737955198873
  Std Dev:	 0.24806749635191724
  Minimum:	 0.0
  Maximum:	 1.0


In [44]:
get_nearest(documents, 50, 2)

100%|██████████| 50/50 [00:43<00:00,  1.15it/s]


k = 50, 2-grams:
  Average:	 0.4225943124652582
  Median: 	 0.43518916449448963
  Std Dev:	 0.2517746316019191
  Minimum:	 0.0
  Maximum:	 1.0


In [45]:
get_nearest(documents, 100, 1)

100%|██████████| 100/100 [02:55<00:00,  1.76s/it]


k = 100, 1-grams:
  Average:	 0.4097647442760424
  Median: 	 0.4231529432872259
  Std Dev:	 0.25142784541772223
  Minimum:	 0.0
  Maximum:	 1.0


In [46]:
get_nearest(documents, 100, 2)

100%|██████████| 100/100 [01:26<00:00,  1.15it/s]


k = 100, 2-grams:
  Average:	 0.4448876219362831
  Median: 	 0.4644632084088827
  Std Dev:	 0.25086697138104436
  Minimum:	 0.0
  Maximum:	 1.0


In [47]:
get_nearest(documents, 100, 3)

100%|██████████| 100/100 [01:04<00:00,  1.56it/s]


k = 100, 3-grams:
  Average:	 0.4615459849822856
  Median: 	 0.4836190613826961
  Std Dev:	 0.25548617576268134
  Minimum:	 0.0
  Maximum:	 1.0


In [48]:
get_nearest(documents, 100, 4)

100%|██████████| 100/100 [00:35<00:00,  2.81it/s]


k = 100, 4-grams:
  Average:	 0.45007878944561464
  Median: 	 0.47191024248581487
  Std Dev:	 0.25499221909463593
  Minimum:	 0.0
  Maximum:	 1.0


In [49]:
get_nearest(documents, 100, 8)

100%|██████████| 100/100 [00:06<00:00, 15.15it/s]


k = 100, 8-grams:
  Average:	 0.4024938471260753
  Median: 	 0.41284141862026763
  Std Dev:	 0.2580207063194688
  Minimum:	 0.0
  Maximum:	 1.0


# Разбиение на слова, при котором разделителем является пробел:

In [50]:
get_nearest(documents, 100, 1, False)

100%|██████████| 100/100 [01:15<00:00,  1.33it/s]


k = 100, 1-grams:
  Average:	 0.4111248420546784
  Median: 	 0.4241579949254954
  Std Dev:	 0.25075850150325424
  Minimum:	 0.0
  Maximum:	 1.0


In [51]:
get_nearest(documents, 100, 2, False)

100%|██████████| 100/100 [00:43<00:00,  2.32it/s]


k = 100, 2-grams:
  Average:	 0.4156093300204595
  Median: 	 0.4272729481704639
  Std Dev:	 0.2552792532818914
  Minimum:	 0.0
  Maximum:	 1.0


In [52]:
get_nearest(documents, 100, 3, False)

100%|██████████| 100/100 [00:19<00:00,  5.22it/s]


k = 100, 3-grams:
  Average:	 0.4098429204030989
  Median: 	 0.42592379742262093
  Std Dev:	 0.2548322846644521
  Minimum:	 0.0
  Maximum:	 1.0


In [53]:
get_nearest(documents, 100, 8, False)

100%|██████████| 100/100 [00:01<00:00, 84.95it/s]


k = 100, 8-grams:
  Average:	 0.35380176941904395
  Median: 	 0.35031538367746595
  Std Dev:	 0.25489870568296447
  Minimum:	 0.0
  Maximum:	 1.0


### 2. SimHash (+1 балл)
По аналогии с пунктом 3, реализуйте предложенный на лекции алгоритм Random Hyperplane SimHash. Если алгоритм реализован правильно, то суммарное количество ноликов и единичек в хешах должно быть примерно одинаковое.

Степень «похожести» двух документов в данном случае будет доля совпавших бит в хеше. Для поиска ближайших по расстоянию Хэмминга хешей используйте приём с разбиением хеша на части.

Оцените качество подбора дубликатов для различных размеров хеша (n). Например, для n равных 16, 64, 128, 256. Для оценки качества используйте ту же метрику, что и в пункте 3.

Для самопроверки, посчитайте сколько получилось нулей и единиц суммарно во всех хешах коллекции. Если алгоритм реализован правильно, то их количество должно быть примерно одинаковое.

In [ ]:
#
#
#
#
# code here
#
#
#
#

### 3. Расширенная нормализация (+0.5 балла за пункт)

#### 3.1
Предложите и реализуйте «нормализацию» слов документа, которая устойчива к переименованию именованных сущностей документа. Проверьте на сколько увеличилась полнота срабатывания пункта 1, а также качество пунктов 3 и 4 с новой нормализацией.

In [54]:
def dupl(docs):  
    documents = dict()
    hashes = dict()
    for doc, content in docs.items():
        encoding = chardet.detect(content)['encoding']
        documents[doc] = normalize(content.decode(encoding))
        hash_doc = md5(" ".join(norm(nltk.word_tokenize(documents[doc]))).encode('utf-8')).hexdigest()
        if hash_doc in hashes:
            hashes[hash_doc].append(doc)
        else:
            hashes[hash_doc] = [doc]
    
    duplicates = []
    sizes = []
    for value in hashes.values():
        if len(value) > 1:
            duplicates.append(value)
            sizes.append(len(value))
    
    print(np.mean(sizes))
    print(np.min(sizes))
    print(np.max(sizes))
    print(np.unique(sizes))
    print(len(sizes))
    print(len(hashes))
    
    pairs = []
    for d in duplicates:
        for i in range(len(d)):
            for j in range(len(d)):
                if i != j:
                    pairs.append((d[i], d[j]))
    
    for first, second in pairs:
        assert ground_truth[first][second] == 1
        ground_truth[first].pop(second)
    
    for d in duplicates:
        for doc in d[1:]:
            del docs[doc]
            del documents[doc]
            del ground_truth[doc]
    
    print("токен является отдельным словом:")
    get_nearest(documents, 100, 3, normlz=True)
    print("разделителем является пробел:")
    get_nearest(documents, 100, 3, tokenize=False, normlz=True)
    print("В тексте только буквы и цифры:")
    get_nearest(documents, 100, 3, normlz=True, delim=True)
    get_nearest(documents, 100, 8, tokenize=False, normlz=True, delim=True)

In [55]:
dupl(docs)

  0%|          | 6/5002 [00:00<01:29, 56.11it/s]

2.0
2
2
[2]
31
5002
токен является отдельным словом:


100%|██████████| 100/100 [00:31<00:00,  3.15it/s]


k = 100, 3-grams:
  Average:	 0.23875577619768476
  Median: 	 0.18487763283997907
  Std Dev:	 0.24269697287019365
  Minimum:	 0.0
  Maximum:	 1.0


  0%|          | 9/5002 [00:00<01:04, 76.83it/s]

...

В тексте только буквы и цифры:


100%|██████████| 100/100 [00:56<00:00,  1.78it/s]


k = 100, 3-grams:
  Average:	 0.18373095228152148
  Median: 	 0.07473906057032167
  Std Dev:	 0.22126358291820622
  Minimum:	 0.0
  Maximum:	 1.0


100%|██████████| 100/100 [00:08<00:00, 12.16it/s]


k = 100, 8-grams:
  Average:	 0.18353736329644063
  Median: 	 0.0768487560649219
  Std Dev:	 0.22186851969415755
  Minimum:	 0.0
  Maximum:	 1.0


#### 3.2 
Предложите и реализуйте удаление слов документа, которые не влияют на логику документа. Проверьте на сколько увеличилась полнота срабатывания пункта 1, а также качество пунктов 3 и 4 с новой нормализацией.

In [61]:
def dupl1(docs):  
    documents = dict()
    hashes = dict()
    for doc, content in docs.items():
        encoding = chardet.detect(content)['encoding']
        documents[doc] = normalize(content.decode(encoding))
        hash_doc = md5(re.sub('[\w\d\s]+', '', documents[doc]).encode('utf-8')).hexdigest()
        if hash_doc in hashes:
            hashes[hash_doc].append(doc)
        else:
            hashes[hash_doc] = [doc]
    
    duplicates = []
    sizes = []
    for value in hashes.values():
        if len(value) > 1:
            duplicates.append(value)
            sizes.append(len(value))
    
    print(np.mean(sizes))
    print(np.min(sizes))
    print(np.max(sizes))
    print(np.unique(sizes))
    print(len(sizes))
    print(len(hashes))
    
    pairs = []
    for d in duplicates:
        for i in range(len(d)):
            for j in range(len(d)):
                if i != j:
                    pairs.append((d[i], d[j]))
    
    for first, second in pairs:
        assert ground_truth[first][second] == 1
        ground_truth[first].pop(second)
    
    for d in duplicates:
        for doc in d[1:]:
            del docs[doc]
            del documents[doc]
            del ground_truth[doc]
    
    get_nearest(documents, 100, 3, remove_words=True)
    get_nearest(documents, 100, 8, remove_words=True)

In [62]:
dupl1(docs)

  0%|          | 8/4982 [00:00<01:13, 67.63it/s]

2.0
2
2
[2]
14
4982


100%|██████████| 100/100 [02:13<00:00,  1.34s/it]


k = 100, 3-grams:
  Average:	 0.49766042156120943
  Median: 	 0.5255591980701877
  Std Dev:	 0.27273541916306887
  Minimum:	 0.0
  Maximum:	 1.0


100%|██████████| 100/100 [00:14<00:00,  6.69it/s]


k = 100, 8-grams:
  Average:	 0.5153265883959712
  Median: 	 0.5432221396750864
  Std Dev:	 0.2718901231204373
  Minimum:	 0.0
  Maximum:	 1.0
